# Recommender Movie System Using Content Filtering Method

In [21]:
#Import The libaries to be used

import pandas as pd
import numpy as np

In [22]:
#Reading the CSV files
df1 = pd.read_csv('data/tmdb_5000_credits.csv')
df2 = pd.read_csv('data/tmdb_5000_movies.csv')
df1

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [44]:
df1.columns =['id','title','genre','keywords']
df2 = df2.merge(df1, on = 'id')


# Data-preprocessing 

In [35]:
columns_to_keep= ['id','original_title','genres','overview']
df3 = df2[columns_to_keep]  
        

In [43]:
missing_values = df3.isnull().sum()
final_pd = df3.dropna()
final_pd.isnull().sum()

id                0
original_title    0
genres            0
overview          0
dtype: int64

## 1.1 Content Based Filtering

In [49]:
#Using Tfid to vectorize our overview with movie titles
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(final_pd['overview'])
tfidf_matrix.shape

(4800, 20978)

### 1.2 Cosine similarity 

In [71]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarity= linear_kernel(tfidf_matrix,tfidf_matrix)
cosine_similarity.shape


(4800, 4800)

In [70]:
indices = pd.Series(final_pd.index, index=final_pd['original_title'])


In [69]:
def recommendation_system(original_title, cosine_similarity=cosine_similarity):
    Movie_index = indices[original_title]
    sim_score = cosine_similarity[Movie_index]
    movie_sim_score = list(enumerate(sim_score))

#Sorting
    Sorted_movies = sorted(movie_sim_score, key=lambda x: x[1], reverse= True)
    Top_movies = [index for index, score in Sorted_movies if index != Movie_index][:10]
    return final_pd['original_title'].iloc[Top_movies]


In [72]:
original_title = 'Titanic'

Top_movies  = recommendation_system(original_title)
print(Top_movies)

1269                                  Raise the Titanic
2143                                         Ghost Ship
2287                         I Can Do Bad All By Myself
770                                       Event Horizon
4287                                            Niagara
3212                                           The Rose
2902                                           Triangle
4228                        The Ballad of Jack and Rose
171     Master and Commander: The Far Side of the World
104                                            Poseidon
Name: original_title, dtype: object
